<h1 style="text-align: center;">Proyect 2 - Stock Forecasting with RNNs, LSTMs and GRU</h1>

<h4 style="text-align: center;">

Roi Jared Flores Garza Stone

Esteban Gomez Valerio

Ivan Morales Camacho

Rafael Takata Garcia

Machine Learning - O2025_MAF3654H

Ing. Juan Antonio Vega Fernández, M. Sc., M. T. Ed

ITESO
</h4>

---

### Introduction

Financial markets are inherently complex, noisy, and non-linear systems, making accurate forecasting of asset prices one of the most challenging and valuable problems in quantitative finance. The ability to predict future stock prices or returns, even over a short horizon, is crucial for developing profitable trading strategies, managing risk, and optimizing investment portfolios.

Traditional time series models, such as ARIMA (Autoregressive Integrated Moving Average), rely on strict statistical assumptions (like linearity and stationarity) that often fail to capture the subtle, long-range temporal dependencies and complex non-linear dynamics inherent in high-frequency financial data.

### Objective

This Jupyter Notebook aims to address these challenges by employing modern Recurrent Neural Network (RNN) architectures, specifically Long Short-Term Memory (LSTM) and Gated Recurrent Unit (GRU). In this case, we will use the 

---

### Libraries:

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np

### Import data:

In [12]:
symbol = "ORCL"
data_ticker = yf.Ticker(symbol)
data = data_ticker.history(period="3mo")
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-08-12 00:00:00-04:00,252.142446,257.423275,250.645044,253.420227,9998400,0.0,0.0
2025-08-13 00:00:00-04:00,256.724506,257.223640,242.219663,243.756989,14182100,0.0,0.0
2025-08-14 00:00:00-04:00,244.525652,248.488776,242.149786,244.535645,10285600,0.0,0.0
2025-08-15 00:00:00-04:00,246.492239,250.195819,242.509146,247.849884,11554000,0.0,0.0
2025-08-18 00:00:00-04:00,245.773496,248.728375,243.197964,248.638535,6759900,0.0,0.0
...,...,...,...,...,...,...,...
2025-11-05 00:00:00-05:00,247.910004,252.399994,247.029999,250.309998,11991500,0.0,0.0
2025-11-06 00:00:00-05:00,247.940002,249.070007,239.289993,243.800003,18678300,0.0,0.0
2025-11-07 00:00:00-05:00,239.000000,240.399994,232.350006,239.259995,20969300,0.0,0.0
